### Asignment 2
* Training a Logistic Regression Classifier with CIFAR-10 dataset using PyTorch
* Draw learning curve and confusion matrix
* Find the best hyperparameters
* Plot gradient norm for each learnable parameter

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import random as rand
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
if torch.cuda.is_available():
    avDev = torch.device("cuda")
else:
    avDev = torch.device("cpu")

print(avDev)

cuda


Loading the dataset from CIFAR-10 dataset

In [ ]:
trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

0it [00:00, ?it/s]

170500096it [00:06, 26899990.80it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Intializing the Hyper Parameters

In [ ]:
torch.manual_seed(123)
batch_size = 100
n_iters = 5000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

hidden_layer = 100
learning_rate = 0.01
momentum = 0.5

10


Implementing the Logistic Regression class inherited from NN.Module super class

In [ ]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_layer)
        self.linear2 = nn.Linear(hidden_layer, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
     
    def forward(self, x):
        hiddenlayer = self.relu(self.linear1(x))
        out = self.softmax(self.linear2(hiddenlayer))
        return out

Loading training and testing data to the loader + intializing the model + putting the model & the loss function into the GPU

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

shape = trainset[0][0].numpy().shape
input_dim = shape[0] * shape[1] * shape[2]
output_dim = 10
 
model = LogisticRegressionModel(input_dim, output_dim)

model.to(avDev)
criterion = nn.CrossEntropyLoss().to(avDev)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

Training and Testing the model

In [ ]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         
        #print(images.size()) #torch.Size([100, 1, 28, 28])
        #print(images.view(-1, 28*28).size()) #torch.Size([100, 784])
        images = images.view(-1, shape[0]*shape[1]*shape[2]).to(avDev)
        labels = labels.to(avDev)
         
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)#
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy and confusion matrix        
            correct = 0
            total = 0
            confusion_matrix = np.zeros((10,10))
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                images = images.view(-1, shape[0]*shape[1]*shape[2]).to(avDev)
                 
                # Forward pass only to get logits/output
                outputs = model(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()
                sizee = len(labels)
                for j in range (sizee):
                  confusion_matrix[labels.cpu()[j]][predicted.cpu()[j]] +=1
             
            accuracy = 100. * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
            print ( "confusion matrix = ",confusion_matrix )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Iteration: 500. Loss: 2.274847984313965. Accuracy: 20.889999389648438
confusion matrix =  [[650.   0.   0.   0.   0.  37.   0.   0. 196. 117.]
 [274.   0.   0.   0.   0. 162.   0.   0. 275. 289.]
 [511.   0.   0.   0.   0. 291.   0.   0.  82. 116.]
 [359.   0.   0.   0.   0. 382.   0.   0.  86. 173.]
 [347.   0.   0.   0.   0. 440.   0.   0.  72. 141.]
 [341.   0.   0.   0.   0. 427.   0.   0.  96. 136.]
 [269.   0.   0.   0.   0. 464.   0.   0.  65. 202.]
 [331.   0.   0.   0.   0. 272.   0.   0. 111. 286.]
 [281.   0.   0.   0.   0.  72.   0.   0. 509. 138.]
 [204.   0.   0.   0.   0.  60.   0.   0. 233. 503.]]
Iteration: 1000. Loss: 2.2385003566741943. Accuracy: 22.25
confusion matrix =  [[544.   0.   0.   0.   0.  66.   0.   0. 317.  73.]
 [164.   0.   0.   0.   0. 209.   0.   0. 386. 241.]
 [315.   0.   0.   0.   0. 424.   0.   0. 185.  76.]
 [203.   0.   0.   1.   0. 518.   0.   0. 139. 139.]
 [210.   0.   0.   0.   0. 564.   0.   0. 141.  85.]
 [181.   0.   0.   0.   0. 578.   0

Try to change the hyper parameters values

In [ ]:
torch.manual_seed(123)
batch_size = 128
n_iters = 5120
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

hidden_layer = 512
learning_rate = 0.015625
momentum = 0.7

13


In [ ]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_layer)
        self.linear2 = nn.Linear(hidden_layer, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
     
    def forward(self, x):
        hiddenlayer = self.relu(self.linear1(x))
        out = self.softmax(self.linear2(hiddenlayer))
        return out

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

shape = trainset[0][0].numpy().shape
input_dim = shape[0] * shape[1] * shape[2]
output_dim = 10
 
model = LogisticRegressionModel(input_dim, output_dim)

model.to(avDev)
criterion = nn.CrossEntropyLoss().to(avDev)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

Train and Test again

In [ ]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         
        #print(images.size()) #torch.Size([100, 1, 28, 28])
        #print(images.view(-1, 28*28).size()) #torch.Size([100, 784])
        images = images.view(-1, shape[0]*shape[1]*shape[2]).to(avDev)
        labels = labels.to(avDev)
         
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)#
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy and confusion matrix        
            correct = 0
            total = 0
            confusion_matrix = np.zeros((10,10))
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                images = images.view(-1, shape[0]*shape[1]*shape[2]).to(avDev)
                 
                # Forward pass only to get logits/output
                outputs = model(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()
                sizee = len(labels)
                for j in range (sizee):
                  confusion_matrix[labels.cpu()[j]][predicted.cpu()[j]] +=1
             
            accuracy = 100. * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
            print ( "confusion matrix = ",confusion_matrix )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Iteration: 500. Loss: 2.194092273712158. Accuracy: 26.100000381469727
confusion matrix =  [[468.   0.   0.   0.   3.  41.  58.   0. 279. 151.]
 [ 82.   0.   0.   0.   9. 153. 131.   0. 255. 370.]
 [199.   0.   0.   0.  21. 202. 353.   0. 117. 108.]
 [118.   0.   0.   0.  28. 388. 207.   0.  74. 185.]
 [114.   0.   0.   0.  41. 259. 399.   0.  75. 112.]
 [ 75.   0.   0.   0.  31. 446. 220.   0. 108. 120.]
 [ 53.   0.   0.   0.   9. 252. 495.   0.  52. 139.]
 [103.   0.   0.   0.  53. 216. 208.   0. 101. 319.]
 [101.   0.   0.   0.   3.  98.  21.   0. 563. 214.]
 [ 61.   0.   0.   0.  12.  51.  78.   0. 201. 597.]]
Iteration: 1000. Loss: 2.1790027618408203. Accuracy: 29.420000076293945
confusion matrix =  [[313.   0.   0.   0.   8.  55.  69. 147. 333.  75.]
 [ 37.   0.   0.   0.  24. 111. 236.  69. 257. 266.]
 [ 93.   0.   0.   0.  67. 141. 371. 171. 124.  33.]
 [ 29.   0.   0.   0.  72. 301. 255. 187.  71.  85.]
 [ 44.   0.   0.   0. 182. 131. 412. 126.  70.  35.]
 [ 12.   0.   0.   0. 

Try to change the Hyper prameters values

In [ ]:
torch.manual_seed(123)
batch_size = 1028
n_iters = 10000000
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs)

hidden_layer = 1028
learning_rate = 0.015625
momentum = 0.9

205600


In [ ]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_layer)
        self.linear2 = nn.Linear(hidden_layer, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
     
    def forward(self, x):
        hiddenlayer = self.relu(self.linear1(x))
        out = self.softmax(self.linear2(hiddenlayer))
        return out

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

shape = trainset[0][0].numpy().shape
input_dim = shape[0] * shape[1] * shape[2]
output_dim = 10
 
model = LogisticRegressionModel(input_dim, output_dim)

model.to(avDev)
criterion = nn.CrossEntropyLoss().to(avDev)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

Train and test again

In [ ]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         
        #print(images.size()) #torch.Size([100, 1, 28, 28])
        #print(images.view(-1, 28*28).size()) #torch.Size([100, 784])
        images = images.view(-1, shape[0]*shape[1]*shape[2]).to(avDev)
        labels = labels.to(avDev)
         
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)#
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy and confusion matrix        
            correct = 0
            total = 0
            confusion_matrix = np.zeros((10,10))
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                images = images.view(-1, shape[0]*shape[1]*shape[2]).to(avDev)
                 
                # Forward pass only to get logits/output
                outputs = model(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)
                 
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()
                sizee = len(labels)
                for j in range (sizee):
                  confusion_matrix[labels.cpu()[j]][predicted.cpu()[j]] +=1
             
            accuracy = 100. * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
            print ( "confusion matrix = ",confusion_matrix )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Iteration: 500. Loss: 2.145411968231201. Accuracy: 31.860000610351562
confusion matrix =  [[530.   5.   0.   0.  36.  27.  44.  33. 261.  64.]
 [109.  59.   0.   0.  61.  95. 120.  28. 238. 290.]
 [219.   9.   0.   0. 267. 117. 198.  63.  90.  37.]
 [129.  10.   0.   0. 155. 306. 144.  90.  73.  93.]
 [123.   5.   0.   0. 387.  93. 232.  58.  66.  36.]
 [ 82.   6.   0.   0. 163. 407. 150.  78.  74.  40.]
 [ 50.   9.   0.   0. 156. 111. 488.  89.  37.  60.]
 [109.   9.   0.   0. 228.  97. 115. 207.  94. 141.]
 [154.   7.   0.   0.  21.  75.  18.   8. 597. 120.]
 [ 99.  12.   0.   0.  36.  44.  68.  31. 199. 511.]]
Iteration: 1000. Loss: 2.09409236907959. Accuracy: 37.119998931884766
confusion matrix =  [[506.  58.  38.   0.  17.  13.  38.  46. 230.  54.]
 [ 82. 338.  11.   0.  26.  55.  83.  28. 159. 218.]
 [157.  44. 159.   0. 173. 104. 201.  63.  77.  22.]
 [ 95.  63.  91.   0.  83. 297. 166.  73.  60.  72.]
 [ 90.  32.  79.   0. 350.  83. 213.  74.  56.  23.]
 [ 53.  35.  90.   0.  8